In [0]:
#TensorBoard on colab
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# Google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install mne

from collections import OrderedDict
import numpy as np
import tensorflow as tf
from datetime import datetime
from sklearn.metrics import cohen_kappa_score
from mne.io import read_raw_edf, find_edf_events
# from flip_gradient import flip_gradient
# import helper_functions as hf
# from load_data import BCICompetition4Set2A
import mne

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
# Data helper functions
test_subject = 9
test_subject -= 1

def BCI_read_data(filename, labels_filename):    
    train_loader = BCICompetition4Set2A(filename=filename, labels_filename=labels_filename)
    train_cnt = train_loader.load()
    train_cnt = train_cnt.drop_channels(['STI 014', 'EOG-left', 'EOG-central', 'EOG-right'])
    assert len(train_cnt.ch_names) == 22
    
    train_cnt = hf.mne_apply(lambda a: a * 1e6, train_cnt)
    train_cnt = hf.mne_apply(lambda a: hf.bandpass_cnt(a, 0, 38.0, train_cnt.info['sfreq'],
                               filt_order=3,
                               axis=1), train_cnt)
    train_cnt = hf.mne_apply(lambda a: hf.exponential_running_standardize(a.T, factor_new=1e-3,
                                                  init_block_size=1000,
                                                  eps=1e-4).T, train_cnt)
    
    print("$$$$$$$$$$$$$$$$$$")
    print(train_cnt.info['sfreq'])
    print("$$$$$$$$$$$$$$$$$$")
    
    
    return train_cnt

def BCI_load_all_data():
    ival = [-500, 4000]
    marker_def = OrderedDict([('Left Hand', [1]), ('Right Hand', [2],),
                              ('Foot', [3]), ('Tongue', [4])])
    
    raw_files = [BCI_read_data("A0" + str(f) + "T.gdf", 
                               "A0" + str(f) + "T.mat") for f in range(1, 10)]
    
    
    train_set_list = [hf.create_signal_target_from_raw_mne(raw, marker_def, ival) for raw in raw_files]
    
    train_set_X = np.concatenate([train_set_list[i].X for i in  range(9) if i!=test_subject])
    # train_set_X = np.concatenate([train_set.X for train_set in train_set_list[:-1]])
    train_set_y = np.concatenate([train_set_list[i].y for i in  range(9) if i!=test_subject])
    # train_set_y = np.concatenate([train_set.y for train_set in train_set_list[:-1]])
    train_set_z = np.concatenate([np.zeros((288)) + i for i in range(8)])
    
    indices = np.arange(train_set_y.shape[0])
    np.random.shuffle(indices)
    
    train_set = hf.SignalAndTarget(train_set_X[indices, :, :], train_set_y[indices], train_set_z[indices])
    test_set = train_set_list[test_subject]
    train_set, valid_set = hf.split_into_two_sets(train_set, first_set_fraction=1-0.2)
    
    iterator = hf.CropsFromTrialsIterator(batch_size=60, input_time_length=1125, n_preds_per_input=4)
    
    return iterator, train_set, valid_set, test_set, raw_files[0].info['sfreq']

In [0]:
iterator, train_set, valid_set, test_set, s = BCI_load_all_data()

In [0]:
def shallow_brain_model(features, mode):
    conv1 = tf.layers.conv2d(inputs=features, filters = 40, kernel_size=(1, 25), strides=1, activation=None)

    conv2 = tf.layers.conv2d(inputs=conv1, filters = 40, kernel_size=(18, 1), strides=1, activation=None)
    batch2 = tf.layers.batch_normalization(conv2, momentum=0.1)
    activation2 = tf.nn.elu(batch2)
    pool2 = tf.layers.max_pooling2d(inputs=activation2, pool_size=(1, 80), strides=15)
    dropout2 = tf.layers.dropout(pool2, 0.5, training=mode)

    conv3 = tf.layers.conv2d(inputs=dropout2, filters = 4, kernel_size=(1, 69), strides=1, activation=None)
    logits = tf.reshape(conv3, [-1, 4])

    return logits

def detect_subject_model(features):
    print(features.shape)
    feat = flip_gradient(features)

    conv1 = tf.layers.conv2d(inputs=feat, filters = 40, kernel_size=(1, 25), strides=1, activation=None)

    conv2 = tf.layers.conv2d(inputs=conv1,filters = 40, kernel_size=(18, 1), strides=1, activation=None)
    batch2 = tf.layers.batch_normalization(conv2, momentum=0.1)
    activation2 = tf.nn.elu(batch2)
    pool2 = tf.layers.max_pooling2d(inputs=activation2, pool_size=(1, 80), strides=15)

    conv3 = tf.layers.conv2d(inputs=pool2, filters = 8, kernel_size=(1, 55), strides=1, activation=None)
    logits = tf.reshape(conv3, [-1, 8])

    print(logits.shape)
    return logits
  
def embed_model(features):
    conv1 = tf.layers.conv2d(inputs=features, filters=1, kernel_size=(3, 100), strides=1, activation=None)
    batch1 = tf.layers.batch_normalization(conv1, momentum=0.1)
    activation1 = tf.nn.elu(batch1)

    conv2 = tf.layers.conv2d(inputs=activation1, filters=1, kernel_size=(3, 100), strides=1, activation=None)
    batch2 = tf.layers.batch_normalization(conv2, momentum=0.1)
    activation2 = tf.nn.elu(batch2)
    
    conv3t = tf.layers.conv2d_transpose(inputs=activation2, filters=1, kernel_size=(3, 100), strides=1, activation=None)
    batch3 = tf.layers.batch_normalization(conv3t, momentum=0.1)
    activation3 = tf.nn.elu(batch3)
    
    conv4t =tf.layers.conv2d_transpose(inputs=activation3, filters=1, kernel_size=(3, 100), strides=1, activation=None)
    batch4 = tf.layers.batch_normalization(conv4t, momentum=0.1)
    activation4 = tf.nn.elu(batch4)
    
    return [activation2, activation4]

In [0]:
# Remove tensorboard files from previous runs 
# import shutil
# shutil.rmtree("./log") 

# Reset graph
tf.reset_default_graph()

# Placeholders
# input_data = tf.placeholder(tf.float32, shape=[60, 22, 1125, 1])

input_data = tf.placeholder(tf.float32, shape=[None, None, None, 1])
rec_l = tf.placeholder(tf.float32, shape=[None, None, None, 1])
labels = tf.placeholder(tf.int32, shape=[None])
mode = tf.placeholder(tf.bool)
lr = tf.placeholder(tf.float32)


# Add new models to the graph
# embedding_model = embed_model(input_data)
# detection_model = detect_subject_model(embedding_model[0])
# class_model = shallow_brain_model(embedding_model[0], mode)
class_model = shallow_brain_model(input_data, mode)
# reconstruction = embedding_model[1]


print(np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))

if tf.train.get_global_step() is None:
    tf.train.create_global_step()

In [0]:
# subject_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=detection_model)
class_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=class_model) 
# reconstruction_loss = tf.losses.mean_squared_error(labels=rec_l, predictions=reconstruction)

# subject_loss_summ = tf.summary.scalar("Subject_loss", subject_loss)
class_loss_summ = tf.summary.scalar("Class_loss", class_loss)
# reconstruction_loss_summ = tf.summary.scalar("Reconstruction_loss", reconstruction_loss)

class_optimizer = tf.train.AdamOptimizer(learning_rate=lr)
# subject_optimizer = tf.train.AdamOptimizer(learning_rate=lr)
# reconstruction_optimizer = tf.train.AdamOptimizer(learning_rate=lr)

# subject_op = subject_optimizer.minimize(loss=subject_loss, global_step=tf.train.get_global_step())
class_op = class_optimizer.minimize(loss=class_loss)
# reconstruction_op = reconstruction_optimizer.minimize(loss=reconstruction_loss)

In [0]:
epoch_number = 30
max_test_acc = 0
min_val_loss = 100

with tf.Session() as sess:
#     writer_train = tf.summary.FileWriter(LOG_DIR + "/train")
#     writer_train.add_graph(sess.graph)
    
#     writer_val = tf.summary.FileWriter(LOG_DIR + "/val")
#     writer_val.add_graph(sess.graph)
    

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()

    # Restore checkpoints
#     try:
#         saver.restore(sess, "/content/gdrive/My Drive/checkpoints/last_term_model/" + str(test_subject+1) + "/" + str(27) + "/model.ckpt")
#     except Exception as e:
#         print (e)

    print("{} Start training...".format(datetime.now()))

    for epoch in range(epoch_number):

        all_loss_rec = []
        val_loss_rec = []
        
        all_pred_class = []
        all_loss_class = []
        all_targets = []
        val_pred_class = []
        val_loss_class = []
        val_targets = []
        test_pred_class = []
        test_targets = []
        
        all_pred_subject = []
        all_loss_subject = []
        all_subjects = []
        val_pred_subject = []
        val_loss_subject = []
        val_subjects = []
        test_pred_subject = []
        
        batch_generator = iterator.get_batches(train_set, shuffle=True)
        batch_generator_2 = iterator.get_batches(valid_set, shuffle=True)
        batch_generator_3 = iterator.get_batches(test_set, shuffle=True)
        
        for inputs, targets, subjects in batch_generator:
# #             summ_rec, _, rec_loss, rec = sess.run([reconstruction_loss_summ, reconstruction_op, reconstruction_loss, reconstruction], feed_dict={input_data: inputs, lr: 0.0005, rec_l: inputs})
#             _, rec_loss, rec = sess.run([reconstruction_op, reconstruction_loss, reconstruction], feed_dict={input_data: inputs, lr: 0.0005, rec_l: inputs})

#             all_loss_rec.append(rec_loss)
            
#             summ_class, _, c_loss, class_logits = sess.run([class_loss_summ, class_op, class_loss, class_model], feed_dict={input_data: inputs, mode:True, lr:0.001, labels:targets})
            _, c_loss, class_logits = sess.run([class_op, class_loss, class_model], feed_dict={input_data: inputs, mode:True, lr:0.001, labels:targets})
            
            prediction = np.argmax(class_logits, 1)
            all_pred_class.extend(prediction)
            all_loss_class.append(c_loss)
            all_targets.extend(targets)
            
# #             summ_subject, _, s_loss, subject_logits = sess.run([subject_loss_summ, subject_op, subject_loss, detection_model], feed_dict={input_data: inputs, lr:0.00005, labels:subjects})
#             _, s_loss, subject_logits = sess.run([subject_op, subject_loss, detection_model], feed_dict={input_data: inputs, lr:0.00005, labels:subjects})

#             if tf.train.get_global_step().eval() % 31 == 0:
#                 writer_train.add_summary(summ_rec, epoch)
#                 writer_train.add_summary(summ_class, epoch)
#                 writer_train.add_summary(summ_subject, epoch)
#                 writer_train.flush()
            
#             prediction = np.argmax(subject_logits, 1)
#             all_pred_subject.extend(prediction)
#             all_loss_subject.append(s_loss)
#             all_subjects.extend(subjects)

        class_accuracy = np.sum(np.equal(all_pred_class, all_targets)) / float(len(all_pred_class))
#         subject_accuracy = np.sum(np.equal(all_pred_subject, all_subjects)) / float(len(all_pred_subject))

        num_nodes = len([n.name for n in tf.get_default_graph().as_graph_def().node])

        print("Epoch {} nodes {}".format(epoch, num_nodes))
        print("Class:    Loss {:.6f}, Accuracy: {}, Kappa {}".format(np.mean(all_loss_class), class_accuracy, cohen_kappa_score(all_pred_class, all_targets)))
#         print("Subjects: Loss {:.6f}, Accuracy: {}, Kappa {}".format(np.mean(all_loss_subject), subject_accuracy, cohen_kappa_score(all_pred_subject, all_subjects)))
#         print("Reconstruction: Loss {:.6}".format(np.mean(all_loss_rec)))

        for inputs, targets, subjects in batch_generator_2:
# #             summ_rec, rec_loss, rec = sess.run([reconstruction_loss_summ, reconstruction_loss, reconstruction], feed_dict={input_data: inputs ,rec_l: inputs})
#             rec_loss, rec = sess.run([reconstruction_loss, reconstruction], feed_dict={input_data: inputs ,rec_l: inputs})
            
#             val_loss_rec.append(rec_loss)
            
#             summ_class, c_loss, class_logits = sess.run([class_loss_summ, class_loss, class_model], feed_dict={input_data: inputs, mode:False, labels:targets})
            c_loss, class_logits = sess.run([class_loss, class_model], feed_dict={input_data: inputs, mode:False, labels:targets})

            prediction = np.argmax(class_logits, 1)
            val_pred_class.extend(prediction)
            val_loss_class.append(c_loss)
            val_targets.extend(targets)
            
# #             summ_subject, s_loss, subjects_logits = sess.run([subject_loss_summ, subject_loss, detection_model], feed_dict={input_data: inputs, labels:subjects})
#             s_loss, subjects_logits = sess.run([subject_loss, detection_model], feed_dict={input_data: inputs, labels:subjects})
            
#             if len(val_loss_subject) == 0:
#                 writer_val.add_summary(summ_rec, epoch)
#                 writer_val.add_summary(summ_class, epoch)
#                 writer_val.add_summary(summ_subject, epoch)
#                 writer_val.flush()
            
#             prediction = np.argmax(subjects_logits, 1)
#             val_pred_subject.extend(prediction)
#             val_loss_subject.append(s_loss)
#             val_subjects.extend(subjects)

        class_accuracy = np.sum(np.equal(val_pred_class, val_targets)) / float(len(val_pred_class))
#         subject_accuracy = np.sum(np.equal(val_pred_subject, val_subjects)) / float(len(val_pred_subject))

        if min_val_loss > np.mean(val_loss_class):
            min_val_loss = np.mean(val_loss_class)
          
        print("Class:    Loss {:.6f}, min_loss: {}, Accuracy: {}, Kappa: {}".format(np.mean(val_loss_class), min_val_loss, class_accuracy, cohen_kappa_score(val_pred_class, val_targets)))
#         print("Subjects: Loss {:.6f}, Accuracy: {}, Kappa: {}".format(np.mean(val_loss_subject), subject_accuracy, cohen_kappa_score(val_pred_subject, val_subjects)))
#         print("Reconstruction: Loss {:.6}".format(np.mean(val_loss_rec)))

        for inputs, targets, subjects in batch_generator_3:
            c_loss, class_logits = sess.run([class_loss, class_model], feed_dict={input_data: inputs, mode:False, labels:targets})

            prediction = np.argmax(class_logits, 1)
            test_pred_class.extend(prediction)
            test_targets.extend(targets)

        class_accuracy = np.sum(np.equal(test_pred_class, test_targets)) / float(len(test_pred_class))

        if class_accuracy > max_test_acc:
            max_test_acc = class_accuracy
            
        print("TestC:    Loss {:.6f}, Accuracy: {}, Max acc: {}, Kappa: {}".format(c_loss, class_accuracy, max_test_acc, cohen_kappa_score(test_pred_class, test_targets)))

#         Save checkpoints
#         saver.save(sess, "/content/gdrive/My Drive/checkpoints/baseline_model/" + str(test_subject+1) + "/" + str(epoch) + "/model.ckpt" )
        
    print("{} Done training...".format(datetime.now()))



In [0]:
batch_generator = iterator.get_batches(train_set, shuffle=True)
count = 0
        
for inputs, targets, subjects in batch_generator:
  count += inputs.shape[0]
  
count


In [0]:
a = []
for i in range(4):
  a.append(np.load("b_" + str(i) + ".npy"))
  
a[0].shape


b = []
for i in range(4):
  b.append(np.load("g_" + str(i) + ".npy"))


In [0]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 4, figsize=(16, 8))

# plt.grid(True)

montage = mne.channels.read_montage('biosemi128')
montage.selection = montage.selection[:22]

info = mne.create_info(montage.ch_names[:22], 250.0, 'eeg', montage=montage)

# for i in range(4):
mne.viz.plot_topomap(a[0].squeeze()[:, 500], pos=info, vmin=-1, vmax=1, show=False, axes=ax[0][0])
mne.viz.plot_topomap(a[1].squeeze()[:, 500], pos=info, vmin=-1, vmax=1, show=False, axes=ax[0][1])
mne.viz.plot_topomap(a[2].squeeze()[:, 500], pos=info, vmin=-1, vmax=1, show=False, axes=ax[0][2])
mne.viz.plot_topomap(a[3].squeeze()[:, 500], pos=info, vmin=-1, vmax=1, show=False, axes=ax[0][3])

d = mne.viz.plot_topomap(b[0].squeeze()[:, 500], pos=info, vmin=-1, vmax=1, show=False, axes=ax[1][0])
mne.viz.plot_topomap(b[1].squeeze()[:, 500], pos=info, vmin=-1, vmax=1, show=False, axes=ax[1][1])
mne.viz.plot_topomap(b[2].squeeze()[:, 200], pos=info, vmin=-1, vmax=1, show=False, axes=ax[1][2])
mne.viz.plot_topomap(b[3].squeeze()[:, 200], pos=info, vmin=-1, vmax=1, show=False, axes=ax[1][3])

# fig.colorbar(d, ax=ax[1][0])

plt.show

In [0]:
test_d = None
batch_generator = iterator.get_batches(train_set, shuffle=True)
batch_generator_2 = iterator.get_batches(valid_set, shuffle=True)
batch_generator_3 = iterator.get_batches(test_set, shuffle=True)

for inputs, targets, subjects in batch_generator:
  test_d = inputs[0, :, :, 0]

In [0]:
a = tf.global_variables()[:6]

a.extend((tf.global_variables()[8:10]))
a

In [0]:
np.max(test_d[:, 1000])
np.min(test_d[:, 1000])